<a href="https://colab.research.google.com/github/apr0vleptos/MultiDiProject/blob/master/report.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# LSH Algorith with Cosine Document similarity

The part of the Locality-Sensitive-Hashing Algorithm consists of 3 parts,as we can see in the picture below:


*   Turn each documment in sets of k-shinglings.
*   Through, the MinHashing Algorithm we turn the above sets into Signatures.
*   Finally, with LSH we hash signatures into backets.


![lsh_diagramm](https://miro.medium.com/max/1904/1*27nQOTC79yfh5lzmL06Ieg.png)

Finally, the candidate pairs (output of LSH) are the input in our Cosine Similarity Algorithm, in order to find the percentage of similarity of document pairs. 

Our implementation is in Python 3. The pilars that are mentioned above, are all implemented seperatly (functional). Then we combine all of the above, in our main script. 




# k-Shingling

The shingling.py file contains 3 functions, that will be descibed below. 

In [0]:
# Function that breaks a text into k-shinglings and returns 
# a list of the MD5 values of each shingling.
def get_shingles(text, k):
    list = [md5(text[i:i + k].encode('ascii')).hexdigest()
            for i in range(len(text) - k)]
    unique_list = reduce(
        lambda x, y: x + [y] if x == [] or x[-1] != y else x, list, [])
    return unique_list

In [0]:
# Function that decides the K, depending the text.
def shingles(text):
    if len(text) > 1000:
        return get_shingles(text, 10)
    return get_shingles(text, 5)

In [0]:
# Function that creates a "universe" of shinglings and constructs a 
# matrix of shinglings for each document
def getSinglingMatrix(text_list):
    universe = set()
    for text in tqdm(text_list, "Creating Universe dictionary from each document"):
        for shingle in shingles(text):
            universe.add(shingle)

    universe = sorted(universe)
    print("Universe of shingles got sorted")

    shingle_array = [[1 if shingle in shingles(doc_text) else 0 for doc_text in text_list] for shingle in tqdm(
        universe, "Creating Shingling matrix")]

    print("Shingling Matrix just got created for all documents")
    return shingle_array

# MinHash

The minhash.py file contains 2 functions, that will be descibed below. 

Function that creates a number of hash functions. The produced hash functions are type of:
>$f(x) = (a x + b)\mod k$  


where $a,b$ are random integers (from 0 to 32).

In [0]:
def getHashfunction(num_of_hash_fun, len_of_array):
    return [lambda x: (randint(0, 32) * x + randint(0, 32)) % len_of_array for _ in tqdm(range(num_of_hash_fun), "Creating {} hash functions".format(num_of_hash_fun))]


In [0]:
# Function that turns the Shingling matrix into signatures, with the help 
# of the getHashfunction
def getSignatures(boolean_mat, num_of_hash_fun):
    hash_func_table = getHashfunction(num_of_hash_fun, len(boolean_mat))

    signatures = [[len(boolean_mat)] * (num_of_hash_fun)
                  for _ in range(len(boolean_mat[0]))]

    for index, row in enumerate(boolean_mat):
        for hash_index, hash_func in enumerate(hash_func_table):
            for col_index, column in enumerate(row):
                if column == 1:
                    if hash_func(index) < signatures[col_index][hash_index]:
                        signatures[col_index][hash_index] = hash_func(index)
    return signatures

#Locality Sensitive Hashing

The lsh.py file contains 2 functions, that will be descibed below. 

In [0]:
# Function that hashes sets of signatures into buckets
def filter_buckets(signature_matrix):
    bands_num = 20
    num_of_buckets = 2 * ceil(sqrt(len(signature_matrix)))
    if num_of_buckets <= 100:
        num_of_buckets = 100
    total_bucket_list = []
    for band in tqdm(range(0, len(signature_matrix[0]), 5), "Creating buckets for each band"):
        band_list = [[] for _ in range(num_of_buckets)]
        for doc_idx in range(len(signature_matrix)):
            band_list[abs(hash(tuple(signature_matrix[doc_idx][band:band + 5]))) % num_of_buckets].append(doc_idx)
        total_bucket_list.append(band_list)
    print("Bucket List just got created for each band")
    return total_bucket_list

In [0]:
# Function that creates an upper triangular matrix that stores the number
# of times that 2 documents were on the same bucket.
def calculate_similarity(total_bucket_list, num_of_docs):
    occurance_matrix = [[0] * num_of_docs for _ in range(num_of_docs)]
    for band in total_bucket_list:
        for index, bucket in enumerate(band):
            if bucket:
                for comb in combinations(bucket, 2):
                    occurance_matrix[comb[0]][comb[1]] += 1
                if index == 0:
                    for comb in combinations(sorted(band[index] + band[index + 1]), 2):
                        occurance_matrix[comb[0]][comb[1]] += 1
                elif index == len(band) - 1:
                    for comb in combinations(sorted(band[index - 1] + band[index]), 2):
                        occurance_matrix[comb[0]][comb[1]] += 1
                else:
                    for comb in combinations(sorted(band[index - 1] + band[index] + band[index + 1]), 2):
                        occurance_matrix[comb[0]][comb[1]] += 1
    return occurance_matrix


# Cosine Similarity

The cosim.py file contains a function that implements the algorithm. Our implementation can find calculate the similarities for many files, not just for 2. 

In [0]:
def cosign_sim(text_list):
    unique_words = []
    doc_words = []
    for text in text_list:
        words = re.split(r'\W+', text)
        doc_words.append(words)
        for word in words:
            if word not in unique_words:
                unique_words.append(word)
    unique_words.sort()

    # bazw tis times emfanishs ths kathe lekshs gia kathe arxeio
    doc_appear = []
    for i in range(len(text_list)):
        doc_appear.append([0] * len(unique_words))
        for j in range(len(doc_words[i])):
            for k in range(len(unique_words)):
                if(unique_words[k] == doc_words[i][j]):
                    doc_appear[i][k] += 1
    # briskw thn omoiothta metaksi twn arxeiwn
    doc_abs = []
    for i in range(len(doc_appear)):
        temp = 0
        for j in range(len(doc_appear[i])):
            temp += doc_appear[i][j]**2
        temp = temp**(0.5)
        doc_abs.append(temp)
    similarity = []
    # upologismos similarities
    for i in range(len(text_list)):
        for j in range(i + 1, len(doc_appear)):
            mul = 0
            for k in range(len(doc_appear[i])):
                mul += doc_appear[i][k] * doc_appear[j][k]
            similarity.append(round(100 * mul / (doc_abs[i] * doc_abs[j]), 2))
    return similarity[0]